In [1]:
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes.config as wc
from weaviate.classes.config import Configure
from weaviate.classes.config import ReferenceProperty
from unstructured.partition.auto import partition
from weaviate.util import generate_uuid5
import re
import requests
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
import os
from typing import List, Dict,Tuple
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.classes import query as wvc
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from weaviate import Client
import openai
from dotenv import load_dotenv
import logging
from flair.data import Sentence
from flair.models import SequenceTagger
from fpdf import FPDF
from bs4 import BeautifulSoup
from llama_index.readers.web import BeautifulSoupWebReader
from io import BytesIO
import tempfile
from langchain.document_loaders import WebBaseLoader


c:\Users\dhanu\.conda\envs\idk_gpu\lib\site-packages\pydantic\_internal\_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_base = ""
openai.api_key = ""

weaviate_url = ""
weaviate_key = ""

In [4]:
URL = weaviate_url
APIKEY = weaviate_key

# Connect to your WCD instance
client = weaviate.connect_to_wcs(
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    headers={
        "X-OpenAI-Api-Key": openai.api_key  # Replace with your OpenAI key
    }
)

client.is_ready()

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/.well-known/ready "HTTP/1.1 200 OK"


True

In [5]:
import weaviate.classes.config as wc
from weaviate.classes.config import ReferenceProperty

client.collections.create(
    name="Demo",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai( # specify the vectorizer and model type you're using
        model="ada",
        model_version="002",
        type_="text"
    ),
    generative_config=wc.Configure.Generative.openai(
        model="gpt-4"  # Optional - Defaults to `gpt-3.5-turbo`
    ),

    # Weaviate can infer schema, but it is considered best practice to define it upfront
    properties=[
        wc.Property(name="content", data_type=wc.DataType.TEXT),
        wc.Property(name="keyword", data_type=wc.DataType.TEXT, skip_vectorization=True),
        wc.Property(name="text", data_type=wc.DataType.TEXT),
        wc.Property(name="embeddings", data_type=wc.DataType.NUMBER_ARRAY, skip_vectorization=True),
            wc.Property(name="page_number",  data_type=wc.DataType.TEXT, skip_vectorization=True),

        ])
    


INFO:httpx:HTTP Request: POST https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"


In [ ]:
dir(client)

In [11]:
client.get_meta()

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"


{'grpcMaxMessageSize': 10485760,
 'hostname': 'http://[::]:8080',
 'modules': {'backup-gcs': {'bucketName': 'weaviate-wcs-prod-cust-us-east1-workloads-backups',
   'rootName': '99285617-6cc7-4333-9eed-8a4232634556'},
  'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started',
   'name': 'Generative Search - Anthropic'},
  'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview',
   'name': 'Generative Search - Anyscale'},
  'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html',
   'name': 'Generative Search - AWS'},
  'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat',
   'name': 'Generative Search - Cohere'},
  'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task',
   'name': 'Generative Search - Databricks'},
  'generative-fri

In [12]:
file_path = "paper_ieee.pdf"



In [9]:






from unstructured_ingest.connector.fsspec.s3 import S3AccessConfig, S3WriteConfig, SimpleS3Config
from unstructured_ingest.connector.local import SimpleLocalConfig
from unstructured_ingest.interfaces import (
    ChunkingConfig,
    EmbeddingConfig,
    PartitionConfig,
    ProcessorConfig,
    ReadConfig,
)
from unstructured_ingest.runner import LocalRunner
from unstructured_ingest.runner.writers.base_writer import Writer
from unstructured_ingest.runner.writers.fsspec.s3 import (
    S3Writer,
)

from unstructured_ingest.connector.local import SimpleLocalConfig
from unstructured_ingest.connector.weaviate import (
    SimpleWeaviateConfig,
    WeaviateAccessConfig,
    WeaviateWriteConfig,
)
from unstructured_ingest.interfaces import (
    ChunkingConfig,
    EmbeddingConfig,
    PartitionConfig,
    ProcessorConfig,
    ReadConfig,
)
from unstructured_ingest.runner import LocalRunner
from unstructured_ingest.runner.writers.base_writer import Writer
from unstructured_ingest.runner.writers.weaviate import (
    WeaviateWriter,
)

import os

In [ ]:
chunking_config = ChunkingConfig(
    chunk_elements=True,  # Split document into chunks
    chunking_strategy="by_title",  # Strategy for chunking
    max_characters=8192,  # Maximum chunk size
    combine_text_under_n_chars=1000,  # Combine small chunks
)

# Embedding Configuration
embedding_config = EmbeddingConfig(
    provider="azure-openai",  # Using OpenAI embeddings
    api_key= openai.api_key, # OpenAI API key
    model="text-embedding-ada-002"  # Specific embedding model
)